In [4]:
import rasterio
import rioxarray as rxr
import numpy as np

In [ ]:
data_name = "./datasets/unzip/WTD_2015-01_2015-01/globgm-wtd-20150131.tif"
tif = rasterio.open(data_name)

## Global map of WTD in 1958

`// rasterio.plot.show(tif, title="Water Table Depth (m)")`

![Water Table Depth globally](./wtd.png)

In [8]:
da = rxr.open_rasterio(data_name, masked=True)
da

<xarray.DataArray (band: 1, y: 21600, x: 43200)> Size: 4GB
[933120000 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 173kB 90.0 89.99 89.98 ... -89.98 -89.99 -90.0
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [9]:
da.nbytes

3732480000

In [10]:
da = da.squeeze("band", drop=True)

In [12]:
da.x.all, da.y.all

(<bound method DataArrayAggregations.all of <xarray.DataArray 'x' (x: 43200)> Size: 346kB
 array([-179.995833, -179.9875  , -179.979167, ...,  179.979167,  179.9875  ,
         179.995833], shape=(43200,))
 Coordinates:
   * x            (x) float64 346kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
     spatial_ref  int64 8B 0>,
 <bound method DataArrayAggregations.all of <xarray.DataArray 'y' (y: 21600)> Size: 173kB
 array([ 89.995833,  89.9875  ,  89.979167, ..., -89.979167, -89.9875  ,
        -89.995833], shape=(21600,))
 Coordinates:
   * y            (y) float64 173kB 90.0 89.99 89.98 ... -89.98 -89.99 -90.0
     spatial_ref  int64 8B 0>)

In [11]:
WTD_RES = 30/3600 # arcsec to degree
ERA5_RES = 0.25 # degree
nb_to_agg = int(np.ceil(ERA5_RES/WTD_RES))
da_coarse = da.coarsen(x=nb_to_agg, y=nb_to_agg, boundary="trim").mean()

In [13]:
da_coarse.nbytes, da_coarse.shape

(4147200, (720, 1440))

In [14]:
df = da_coarse.to_dataframe("wtd")
df = df.drop(columns="spatial_ref")
df.iloc[np.where(~np.isnan(df["wtd"]))]

wtd
y       x                 
 83.125 -77.625  -0.000003
        -77.375   7.504607
        -77.125  29.886253
        -76.875  54.301079
        -76.625  41.989902
...                    ...
-55.875 -67.875  25.601067
        -67.625  52.614300
        -67.375  45.208469
        -67.125  43.232643
        -66.875   3.341017

[254350 rows x 1 columns]

In [15]:
ds = da_coarse.to_dataset(name="wtd")
ds

<xarray.Dataset> Size: 4MB
Dimensions:      (x: 1440, y: 720)
Coordinates:
  * x            (x) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * y            (y) float64 6kB 89.87 89.63 89.38 ... -89.37 -89.62 -89.87
    spatial_ref  int64 8B 0
Data variables:
    wtd          (y, x) float32 4MB nan nan nan nan nan ... nan nan nan nan nan

In [16]:
ds.to_dataframe()

spatial_ref  wtd
x        y                        
-179.875  89.875            0  NaN
          89.625            0  NaN
          89.375            0  NaN
          89.125            0  NaN
          88.875            0  NaN
...                       ...  ...
 179.875 -88.875            0  NaN
         -89.125            0  NaN
         -89.375            0  NaN
         -89.625            0  NaN
         -89.875            0  NaN

[1036800 rows x 2 columns]

In [17]:
ds.y.values


array([ 89.875,  89.625,  89.375,  89.125,  88.875,  88.625,  88.375,
        88.125,  87.875,  87.625,  87.375,  87.125,  86.875,  86.625,
        86.375,  86.125,  85.875,  85.625,  85.375,  85.125,  84.875,
        84.625,  84.375,  84.125,  83.875,  83.625,  83.375,  83.125,
        82.875,  82.625,  82.375,  82.125,  81.875,  81.625,  81.375,
        81.125,  80.875,  80.625,  80.375,  80.125,  79.875,  79.625,
        79.375,  79.125,  78.875,  78.625,  78.375,  78.125,  77.875,
        77.625,  77.375,  77.125,  76.875,  76.625,  76.375,  76.125,
        75.875,  75.625,  75.375,  75.125,  74.875,  74.625,  74.375,
        74.125,  73.875,  73.625,  73.375,  73.125,  72.875,  72.625,
        72.375,  72.125,  71.875,  71.625,  71.375,  71.125,  70.875,
        70.625,  70.375,  70.125,  69.875,  69.625,  69.375,  69.125,
        68.875,  68.625,  68.375,  68.125,  67.875,  67.625,  67.375,
        67.125,  66.875,  66.625,  66.375,  66.125,  65.875,  65.625,
        65.375,  65.